# 0. 기본 정보
- 코드 작성일: 2025년 3월 29일
- 코드 작성 목적: 크롤링 실습 겸 논문 제목 및 초록 데이터 수집

In [1]:
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup

In [2]:
import time
import pandas as pd

# 1. DBpia 크롤링

In [3]:
# ChromeDriver 자동 설치 및 실행
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 10)  # 최대 10초 대기

In [4]:
# DBpia 대분류 딕셔너리
category_dict = {
    '인문학':'NA',
    '사회과학':'NB',
    '자연과학':'NC',
    '공학':'ND',
    '의약학':'NE',
    '농수해양학':'NF',
    '예술체육학':'NG',
    '복합학':'NH'
}

In [5]:
# 모든 데이터를 저장할 리스트
all_data = []

In [6]:
# DBpia 사이트 검색 기능 기반 크롤링
for category_name, category_code in category_dict.items():
    try:
        # URL 생성 및 페이지 이동
        url = f'https://www.dbpia.co.kr/search/topSearch?collectionQuery=&filter=&prefix=&subjectCategory={category_code}'
        driver.get(url)
        print(f"\n[{category_name}] 페이지로 이동 중...")
        time.sleep(5)  # 페이지 로드 대기

        try:
            # '이용순' 버튼 클릭
            button = wait.until(EC.element_to_be_clickable((By.ID, "sortByUsedCntButton")))
            button.click()
            print(f"✅ '{category_name}' - '이용순' 버튼 클릭 완료!")
            time.sleep(3)  # 클릭 후 대기
        except Exception as e:
            print(f"❌ '{category_name}' - '이용순' 버튼 클릭 실패: {e}")
            continue  # 다음 카테고리로 이동

        try:
            # '20개씩' 버튼을 클릭하여 드롭다운 메뉴 열기
            select_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".thesisAll__selectTit")))
            select_button.click()
            print(f"✅ '{category_name}' - '20개씩' 드롭다운 메뉴 열기 완료!")
            time.sleep(1)  # 드롭다운 메뉴 열리기 대기

            # '100개씩' 버튼 클릭
            per_page_button = wait.until(EC.element_to_be_clickable((By.ID, "get100PerPage")))
            per_page_button.click()
            print(f"✅ '{category_name}' - '100개씩' 버튼 클릭 완료!")
            time.sleep(5)  # 클릭 후 충분히 대기
        except Exception as e:
            print(f"❌ '{category_name}' - '100개씩' 버튼 클릭 실패: {e}")
            continue  # 다음 카테고리로 이동

        data = []  # 현재 카테고리의 데이터를 저장할 리스트

        # 논문 목록 컨테이너
        containers = driver.find_elements(By.CLASS_NAME, 'thesisWrap')
        print(f"🔍 '{category_name}' - 총 {len(containers)}개의 논문 발견")

        # 현재 탭 저장
        main_window = driver.current_window_handle

        for element in containers:
            try:
                title = element.find_element(By.CLASS_NAME, 'thesis__tit').text
            except:
                title = ''
            try:
                link = element.find_element(By.CLASS_NAME, 'thesis__link').get_attribute('href')
            except:
                link = ''

            if link:
                try:
                    # 새 탭 열기
                    driver.execute_script("window.open('');")
                    driver.switch_to.window(driver.window_handles[1])  # 새 탭으로 전환
                    driver.get(link)
                    time.sleep(3)  # 페이지 로드 대기

                    try:
                        abstract = driver.find_element(By.CLASS_NAME, 'abstractTxt').text
                    except:
                        abstract = ''

                    # 수집된 데이터 저장
                    data.append({
                        '카테고리': category_name,
                        '제목': title,
                        '초록': abstract
                    })
                    print(f"✅ '{category_name}' - 논문 수집 완료: {title}")

                except Exception as e:
                    print(f"❌ '{category_name}' - 논문 수집 중 오류 발생: {e}")

                finally:
                    # 새 탭 닫기
                    driver.close()
                    driver.switch_to.window(main_window)  # 원래 탭으로 전환
                    time.sleep(2)  # 페이지 전환 안정화

        # 카테고리별 데이터 통합 저장
        all_data.extend(data)

    except Exception as e:
        print(f"❌ '{category_name}' - 크롤링 중 오류 발생: {e}")



[인문학] 페이지로 이동 중...
✅ '인문학' - '이용순' 버튼 클릭 완료!
✅ '인문학' - '20개씩' 드롭다운 메뉴 열기 완료!
✅ '인문학' - '100개씩' 버튼 클릭 완료!
🔍 '인문학' - 총 100개의 논문 발견
✅ '인문학' - 논문 수집 완료: 썸타기와 어장관리에 대한 철학적 고찰
✅ '인문학' - 논문 수집 완료: 인공 지능에서 인공 감정으로 : 감정을 가진 기계는 실현가능한가?
✅ '인문학' - 논문 수집 완료: [논문철회] '관음충'의 발생학: 한국남성성의 불완전변태과정(homomorphism)의 추이에 대한 신물질주의적 분석
✅ '인문학' - 논문 수집 완료: 한국의 초저출산: 무엇이 원인이고 무엇이 해법인가
✅ '인문학' - 논문 수집 완료: GMO의 윤리적 문제
✅ '인문학' - 논문 수집 완료: 왜 한국 남성은 한국여성들에게 분노하는가 : 여성혐오, 한국사회가 가지고 있는 어떤 특수성
✅ '인문학' - 논문 수집 완료: 안락사 사례로 보는 생명과 권리의 문제
✅ '인문학' - 논문 수집 완료: 인터넷 밈의 언어적 성격 고찰
✅ '인문학' - 논문 수집 완료: 대학 신입생 글쓰기에 나타난 문장 오류 양상 분석
✅ '인문학' - 논문 수집 완료: 4차 산업 혁명 시대, 대학 교육과 콘텐츠
✅ '인문학' - 논문 수집 완료: 배아복제 기술의 윤리적 문제와 줄기세포 연구의 한계
✅ '인문학' - 논문 수집 완료: 동물실험과 심의
✅ '인문학' - 논문 수집 완료: 강남역 살인사건부터 '메갈리아' 논쟁까지 : '페미니즘 봉기'와 한국 남성성의 위기
✅ '인문학' - 논문 수집 완료: 인공지능은 인간의 일자리를 얼마나 대체할 것인가 : 인공지능 시대의 기술과 노동에 관한 시론
✅ '인문학' - 논문 수집 완료: 썸을 탄다는 것은 무엇인가?: 신조어 “썸타다”의 적용조건 분석
✅ '인문학' - 논문 수집 완료: 니체와 프롬의 철학으로 되짚어 본 EXO의 〈으르렁〉
✅ '인문학' - 논문 수집 완료: 방탄소년단: 새로운 세대의 새로운 소통

In [ ]:
driver.quit()

In [12]:
all_data

[{'카테고리': '인문학',
  '제목': '썸타기와 어장관리에 대한 철학적 고찰',
  '초록': '최근 한국 사회에서 유통되기 시작한 신조어 ‘썸탄다’는 흔히 본격적인 연인관계로 발전하기 이전 단계의 애매한 남녀관계를 뜻하는 것으로 이해된다. 그러나 그것이 정확히 어떤 의미를 지니는지, 썸탄다는 활동의 정체가 정확히 무엇인지는 여전이 오리무중이다. 열 사람에게 물어보면 열 사람 모두 다른 답변을 내놓을 지경이다. 이런 배경 하에서 필자는 미국의 철학자 해리 프랑크푸르트(Harry Frankfurt)의 인간관에 의거하여 의지적 불확정성 개념을 도입하고, 그 개념을 통해서 썸타기의 본성을 포착할 것이다. 필자의 제안은 상대방에 대한 이성적 호감을 지닌 두 남녀가 새롭게 만남을 시작하며 자신들의 의지적 불확정성에 대하여 서로 영향을 주고 받는 과정이 썸타기의 핵심이라는 것이다. 썸타기에 대한 필자의 이러한 제안은 썸타기와 어장관리의 관계에 대한 흥미로운 시각을 제공해 줄 것인데, 그에 대해서는 본 논문의 말미에서 상세히 논의될 것이다.'},
 {'카테고리': '인문학',
  '제목': '인공 지능에서 인공 감정으로 : 감정을 가진 기계는 실현가능한가?',
  '초록': '인공 감정에 관한 철학적 탐구가 필요한 시점이다. 인간의 고유한 영역으로 간주되던 인지적 과제에서 기계의 추월을 염려하는 처지가 되자, 사람들은 이제 이성이 아니라 감정에서 인간의 고유성을 찾으려한다. 하지만 최근 인공지능 로봇에 감성을 불어넣는 작업이 새로운 화두로 떠오르고 있다. 이 글은 인공 감정의 실현가능성과 잠재적인 위험을 논의한다. 먼저, 감성 로봇의 개발 현황과 주요한 동기들을 개괄하고, 왜 로봇의 감정이 문제가 되는지 살펴본다. 진정한 감정-소유 로봇이 가능한지 검토하기 위해 감정을 선험적으로 정의하기보다는 감정이 수행하는 몇 가지 핵심 역할을 소개하고 이로부터 어떤 대상에 감정을 부여할 수 있는 기준들을 제안한다. 나는 이런 기준에 비추어 진정한 감정 로봇이 근미래에 실현될 가

In [13]:
# 크롤링 결과 저장
df = pd.DataFrame(all_data)
df.to_csv('dbpia_crawling_result.csv', encoding='utf-8-sig', index=False)
print("크롤링 완료. csv 파일로 저장했습니다.")

크롤링 완료. csv 파일로 저장했습니다.


In [14]:
# 결과 출력
print(f"총 {len(df)}개의 학술자료 수집 완료.")

총 800개의 학술자료 수집 완료.


In [15]:
df

,카테고리,제목,초록
0,인문학,썸타기와 어장관리에 대한 철학적 고찰,최근 한국 사회에서 유통되기 시작한 신조어 ‘썸탄다’는 흔히 본격적인 연인관계로 발...
1,인문학,인공 지능에서 인공 감정으로 : 감정을 가진 기계는 실현가능한가?,인공 감정에 관한 철학적 탐구가 필요한 시점이다. 인간의 고유한 영역으로 간주되던 ...
2,인문학,[논문철회] '관음충'의 발생학: 한국남성성의 불완전변태과정(homomorphism...,본 논고는 대한민국 사회를 관통하고 있는 불법촬영물이라는 특정 포획물을 기반으로 분...
3,인문학,한국의 초저출산: 무엇이 원인이고 무엇이 해법인가,등록된 정보가 없습니다.
4,인문학,GMO의 윤리적 문제,"본 소고는 현재 유통되고 있는 GMO와 연관된 세 가지 윤리적 문제, 즉 GMO가 ..."
...,...,...,...
795,복합학,언어 폭력 확산과 미디어 : 공격성에 대한 사회 학습 이론과 점화 효과를 중심으로,이 글은 최근 우리 사회에 확산되고 있는 폭력적 말하기 현상을 미디어 효과 측면에서...
796,복합학,배아 유전자 치료 : 왜 이 시점에 쟁점이 되고 있는가,최근 배아의 유전자 치료를 목표로 하는 배아 유전체 편집 연구가 시도되고 있다. 이...
797,복합학,텍스트 마이닝 기법을 이용한 유튜브 추천 알고리즘의 필터버블 현상 분석,이 연구에서는 필터버블 현상의 주요 요인인 추천 알고리즘의 정치적 편향성(추천 알고...
798,복합학,블록체인 기반의 서비스 현황 및 문제점 분석,인터넷과 IT를 활용한 스마트폰의 보편화 및 모바일 쇼핑 비중의 확대로 스마트 결재...


# 9. RISS 인기 학술자료 주제별 제목 수집

In [2]:
# ChromeDriver 자동 설치 및 실행
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [3]:
# RISS 최신인기학술자료 사이트로 이동
url = 'https://www.riss.kr/PopularList.do'
driver.get(url)
time.sleep(3)

In [4]:
# 학문 코드와 이름 매핑
discipline_codes = {
    "10": "인문학",
    "20": "사회과학",
    "30": "자연과학",
    "40": "공학",
    "50": "의약학",
    "60": "농수해양",
    "70": "예술체육",
    "80": "교육",
    "90": "복합학"
}

all_data = []

In [5]:
for code, name in discipline_codes.items():
    try:
        # 학문 클릭
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[title='{name}']"))
        )
        element.click()
        time.sleep(2)

        # "더보기" 버튼 반복 클릭
        while True:
            try:
                more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.ID, "nextList"))
                )
                more_button.click()
                time.sleep(2)
            except:
                print(f"'{name}' 학문의 더보기 버튼이 더 이상 없음.")
                break

        # 학술자료 제목 수집
        titles = driver.find_elements(By.CSS_SELECTOR, "a.title.bold")
        discipline_data = [title.text for title in titles]
        print(f"{name}에서 {len(discipline_data)}개 수집 완료.")
        all_data.extend(discipline_data)

        # 메인 페이지로 돌아가기
        driver.get(url)
        time.sleep(3)

    except Exception as e:
        print(f"{name}에서 에러 발생: {e}")

'인문학' 학문의 더보기 버튼이 더 이상 없음.
인문학에서 99개 수집 완료.
'사회과학' 학문의 더보기 버튼이 더 이상 없음.
사회과학에서 99개 수집 완료.
'자연과학' 학문의 더보기 버튼이 더 이상 없음.
자연과학에서 99개 수집 완료.
'공학' 학문의 더보기 버튼이 더 이상 없음.
공학에서 99개 수집 완료.
'의약학' 학문의 더보기 버튼이 더 이상 없음.
의약학에서 99개 수집 완료.
'농수해양' 학문의 더보기 버튼이 더 이상 없음.
농수해양에서 99개 수집 완료.
'예술체육' 학문의 더보기 버튼이 더 이상 없음.
예술체육에서 99개 수집 완료.
'교육' 학문의 더보기 버튼이 더 이상 없음.
교육에서 99개 수집 완료.
'복합학' 학문의 더보기 버튼이 더 이상 없음.
복합학에서 99개 수집 완료.


In [6]:
# 결과 출력
print(f"총 {len(all_data)}개의 학술자료 수집 완료.")
driver.quit()

총 891개의 학술자료 수집 완료.


In [8]:
print(all_data)

['부끄러움/창피함/쑥스러움/수치스러움/수줍음간의 관계 고찰', '영어와 한국어 언어 유머 비교 분석', 'AI 디지털교과서 도입의 기대와 우려: AI 및 디지털 도구 활용 실태 조사를 기반으로', '한국어교육에서의 대화형 인공지능 챗봇 적용 가능성 탐색 -고급 한국어 학습자와 ChatGPT의 상호작용 분석을 중심으로-', '카카오톡 환경에서의 의사소통 방식 고찰', '아동의 호흡 기능에 대한 진단과 간호사정', '유토피아, 그 개념과 기능', '썸타기와 어장관리에 대한 철학적 고찰', '르네상스 시대 여성의 지위', '크리스퍼 유전자가위에 의한 인간 배아 유전자 편집', '인공지능 시대의 한국어 의미 교육 -AI 번역, 챗봇 빌더, 생성형 AI를 중심으로-', '한강 소설에 나타나는 ‘채식’의 의미', '아동의 언어발달 단계에 관한 심리학적 연구 Ⅰ', '김초엽 소설에 나타난 포스트휴머니즘과 장애', '인공지능(AI)을 활용한 한국어교육 방안 연구', '낭만적 자아와 우주적 사랑', '수렵ㆍ채집 경제에서 농경으로의 轉移과정에 대한 이론적 고찰', '마약중독의 정신의학적 이해', '에릭슨의 사회심리발달 8단계 이론의 분석', '우리가 사는 세상이 가상현실이 아닌 거의 확실한 이유', '원효의 화엄학', '챗GPT를 활용한 시창작 방안 연구', 'MBTI성격유형에 따른 사랑의 구성요소와 사랑유형분석', '일본 애니메이션에 관하여', '중학교 교사의 인공지능 활용 교육에 대한 인식조사', 'MZ세대의 특성과 언어 사용 연구', '한일 영화를 통해 본 ‘은둔형 외톨이’와 ‘히키코모리’ 비교', '좋은 삶이란 무엇인가?', '챗(Chat) GPT를 활용한 대학 교양 읽기-쓰기의 운영 방안 탐색 - 한국외국어대학교 사례를 중심으로 -', '벤야민의 회상 개념과 망각된 시간의 구제', '로베르트 발저의 문학에 나타난 ‘하인 정신’의 형상들 – 1910년경 작품들을 중심으로', "아동학대는 '괴물 같은 부모'의 탓?", '클라이스트의 ｢성 체칠리아 또는 음악의 힘｣에 나타